In [1]:
import sys
import time
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
import matplotlib
matplotlib.use('TKAgg')
from matplotlib import pyplot
import matplotlib.pyplot as plt
from sklearn.metrics import auc, roc_curve
from sklearn import metrics, svm
from sklearn.metrics import precision_recall_fscore_support
import pandas as pd
from sklearn import preprocessing
import warnings
warnings.filterwarnings('ignore')
from sklearn.utils import shuffle
from tensorflow.keras.callbacks import TensorBoard

def dataProc(X1,y1):
    X1 = pd.DataFrame(X1)
    X1=X1.to_numpy()
    y1 = y1.to_numpy()
    
    
    min_max_scaler = preprocessing.MinMaxScaler()
    X1 = min_max_scaler.fit_transform(X1)
    
    X1, y1 = shuffle(X1, y1, random_state=0)
    return X1,y1

## Loading and Processing Data

In [2]:
sysName = "IEEE14"
testType = "ModelsFinalCV"
numEpochs = 100
checkpoint_path = testType+"/Saved Models/"+sysName+"_models/"

X1 = pd.read_csv("Data/"+sysName+"/"+sysName+"Data_40k_lowSparsity"+".csv")
y1 = pd.read_csv("Data/"+sysName+"/"+sysName+"Labels_40k_lowSparsity"+".csv")
X2 = pd.read_csv("Data/"+sysName+"/"+sysName+"Data_40k_midSparsity"+".csv")
y2 = pd.read_csv("Data/"+sysName+"/"+sysName+"Labels_40k_midSparsity"+".csv")
X3 = pd.read_csv("Data/"+sysName+"/"+sysName+"Data_40k_highSparsity"+".csv")
y3 = pd.read_csv("Data/"+sysName+"/"+sysName+"Labels_40k_highSparsity"+".csv")

Xv1 = pd.read_csv("Data/"+sysName+"/"+sysName+"Data_10k_lowSparsity"+".csv")
yv1 = pd.read_csv("Data/"+sysName+"/"+sysName+"Labels_10k_lowSparsity"+".csv")
Xv2 = pd.read_csv("Data/"+sysName+"/"+sysName+"Data_10k_midSparsity"+".csv")
yv2 = pd.read_csv("Data/"+sysName+"/"+sysName+"Labels_10k_midSparsity"+".csv")
Xv3 = pd.read_csv("Data/"+sysName+"/"+sysName+"Data_10k_highSparsity"+".csv")
yv3 = pd.read_csv("Data/"+sysName+"/"+sysName+"Labels_10k_highSparsity"+".csv")

Xt1 = pd.read_csv("Data/"+sysName+"/"+sysName+"Data_4k_lowSparsity"+".csv")
yt1 = pd.read_csv("Data/"+sysName+"/"+sysName+"Labels_4k_lowSparsity"+".csv")
Xt2 = pd.read_csv("Data/"+sysName+"/"+sysName+"Data_4k_midSparsity"+".csv")
yt2 = pd.read_csv("Data/"+sysName+"/"+sysName+"Labels_4k_midSparsity"+".csv")
Xt3 = pd.read_csv("Data/"+sysName+"/"+sysName+"Data_4k_highSparsity"+".csv")
yt3 = pd.read_csv("Data/"+sysName+"/"+sysName+"Labels_4k_highSparsity"+".csv")

X1,y1 = dataProc(X1,y1)
Xv1,yv1 = dataProc(Xv1,yv1)
Xt1,yt1 = dataProc(Xt1,yt1)
X2,y2 = dataProc(X2,y2)
Xv2,yv2 = dataProc(Xv2,yv2)
Xt2,yt2 = dataProc(Xt2,yt2)

X3,y3 = dataProc(X3,y3)
Xv3,yv3 = dataProc(Xv3,yv3)
Xt3,yt3 = dataProc(Xt3,yt3)

numFeatures = len(X1[0])

X = np.concatenate((X1, X2, X3), axis=0)
y = np.concatenate((y1, y2, y3), axis=0)
Xv = np.concatenate((Xv1, Xv2, Xv3), axis=0)
yv = np.concatenate((yv1, yv2, yv3), axis=0)
Xt = np.concatenate((Xt1, Xt2, Xt3), axis=0)
yt = np.concatenate((yt1, yt2, yt3), axis=0)

X, y = shuffle(X, y, random_state=0)
Xv, yv = shuffle(Xv, yv, random_state=0)
Xt, yt = shuffle(Xt, yt, random_state=0)

In [3]:
resultsLow = []
resultsMid = []
resultsHigh = []

## Training

In [4]:
NUM_EPOCHS = 30

In [6]:
num_layers = [3,4,5]
layer_sizes = [64,128,256,512]
dropouts = [0.2, 0.3, 0.4]

for layer in num_layers:
    for layer_size in layer_sizes:
        for dropout in dropouts:
#             NAME = "{}-layers-{}-nodes".format(layer, layer_size)
            LOGNAME = "{}-layers-{}-nodes-{}-dropout-{}".format(layer, layer_size, dropout, int(time.time()))
            tensorboard = TensorBoard(log_dir='logs\AdvancedTest\{}'.format(LOGNAME))
            print(LOGNAME)

            model = keras.Sequential()
            model.add(tf.keras.Input(shape=(numFeatures,)))
            
            model.add(  tf.keras.layers.Dense(layer_size, activation='selu' , 
                                                  kernel_regularizer=keras.regularizers.l2(0.001) )  )
            ##Adding Layers
            for i in range(layer-1):
                model.add(  tf.keras.layers.Dense(layer_size/((i+1)**2), activation='selu' , 
                                                  kernel_regularizer=keras.regularizers.l2(0.001) )  )
                model.add( tf.keras.layers.Dropout(dropout) )
                
            ##Output Layer
            model.add( tf.keras.layers.Dense(1,activation='sigmoid') )

            ##Compiling model
            model.compile(loss='binary_crossentropy',
                         optimizer=keras.optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0),
                         metrics = ['accuracy'])

            ##Training Model
            model.fit(X,y, batch_size = 32,
                      epochs = NUM_EPOCHS,
                      validation_data=(Xv,yv),
                      callbacks = [tensorboard] )

            ##Saving Model
#             model.save('SavedModels\{}.h5'.format(LOGNAME))

            ##Testing Model
            results1 = model.evaluate(Xt1, yt1)
            results2 = model.evaluate(Xt2, yt2)
            results3 = model.evaluate(Xt3, yt3)

            resultsLow.append(results1[1])
            resultsMid.append(results2[1])
            resultsHigh.append(results3[1])
            
writer = pd.ExcelWriter('RESULTS_'+sysName+'_'+testType+'_'+str(numEpochs)+'Epochs.xlsx', engine='xlsxwriter') #CHANGE THE NAME OF THE OUTPUT EXCEL FILE HERE

Results = pd.DataFrame({'Model': names, 'Low': resultsLow,'Mid': resultsMid, 'High': resultsHigh})

Results.to_excel(writer, sheet_name=sysName)

3-layers-64-nodes-0.2-dropout-1573158507
Train on 119997 samples, validate on 29997 samples
Epoch 1/30
119997/119997 [==============================] - 14s 114us/sample - loss: nan - acc: 0.5000 - val_loss: nan - val_acc: 0.4999
Epoch 2/30
119997/119997 [==============================] - 13s 104us/sample - loss: nan - acc: 0.5000 - val_loss: nan - val_acc: 0.4999
Epoch 3/30
119997/119997 [==============================] - 13s 105us/sample - loss: nan - acc: 0.5000 - val_loss: nan - val_acc: 0.4999
Epoch 4/30
119997/119997 [==============================] - 13s 106us/sample - loss: nan - acc: 0.5000 - val_loss: nan - val_acc: 0.4999
Epoch 5/30
119997/119997 [==============================] - 12s 102us/sample - loss: nan - acc: 0.5000 - val_loss: nan - val_acc: 0.4999
Epoch 6/30
119997/119997 [==============================] - 12s 104us/sample - loss: nan - acc: 0.5000 - val_loss: nan - val_acc: 0.4999
Epoch 7/30
119997/119997 [==============================] - 14s 117us/sample - loss: n

119997/119997 [==============================] - 23s 192us/sample - loss: nan - acc: 0.5000 - val_loss: nan - val_acc: 0.4999
Epoch 28/30
119997/119997 [==============================] - 22s 182us/sample - loss: nan - acc: 0.5000 - val_loss: nan - val_acc: 0.4999
Epoch 29/30
119997/119997 [==============================] - 23s 191us/sample - loss: nan - acc: 0.5000 - val_loss: nan - val_acc: 0.4999
Epoch 30/30
3999/3999 [==============================] - 0s 77us/sample - loss: nan - acc: 0.4999
3-layers-64-nodes-0.4-dropout-1573159757
Train on 119997 samples, validate on 29997 samples
Epoch 1/30
119997/119997 [==============================] - 26s 213us/sample - loss: nan - acc: 0.5000 - val_loss: nan - val_acc: 0.4999
Epoch 2/30
119997/119997 [==============================] - 25s 208us/sample - loss: nan - acc: 0.5000 - val_loss: nan - val_acc: 0.4999
Epoch 3/30
119997/119997 [==============================] - 24s 201us/sample - loss: nan - acc: 0.5000 - val_loss: nan - val_acc: 0.49

KeyboardInterrupt: 